### S_Game_Statistics

In [19]:
import pandas as pd
from datetime import datetime
import sqlite3
import hashlib


In [31]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
table_name='S_Game_Statistics'
connection=sqlite3.connect(database=data_base)
cursor = connection.cursor()
drop_query = f"DROP TABLE IF EXISTS {table_name};"
cursor.execute(drop_query)
connection.commit()
connection.close()

In [21]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
connection=sqlite3.connect(database=data_base)
query=f'''

select g.Game_hashkey
     ,t.team
     ,case when g.team=t.team then 1
           else 0
      end Home_flg 
      ,sgts.Disposals
      ,sgts.Kicks
      ,sgts.Marks
      ,sgts.Handballs
      ,sgts.Goals
      ,sgts.Behinds
      ,sgts.HitOuts
      ,sgts.Tackles
      ,sgts.Rebounds
      ,sgts.Inside50
      ,sgts.Clearances
      ,sgts.Clangers
      ,sgts.Frees
      ,sgts.FreesAgainst
      ,sgts.ContestedPossessions
      ,sgts.UncontestedPossessions
      ,sgts.ContestedMarks
      ,sgts.MarksInside50
      ,sgts.OnePercenters
      ,sgts.Bounces
      ,sgts.GoalAssists
from h_game g
join l_game_team lgt
on   g.Game_hashkey=lgt.game_hashkey
join H_team t
on   lgt.team_hashkey=t.team_hashkey
join s_game_team_statistics sgts
on   lgt.GameTeam_Hashkey=sgts.GameTeam_Hashkey
--where g.year=2024
--and   g.round='R01'
'''

df=pd.read_sql(query,connection)
connection.close()
df

,Game_Hashkey,Team,Home_flg,Disposals,Kicks,Marks,Handballs,Goals,Behinds,HitOuts,...,Clangers,Frees,FreesAgainst,ContestedPossessions,UncontestedPossessions,ContestedMarks,MarksInside50,OnePercenters,Bounces,GoalAssists
0,63b4561a7638772cc4a22044d02a583f266de953521036...,CW,1,289,190,93,99,13,12,19,...,39,16,29,103,181,7,12,43,18,11
1,63b4561a7638772cc4a22044d02a583f266de953521036...,RI,0,292,185,83,107,8,15,32,...,40,29,16,122,173,13,14,39,14,4
2,8a951fd81dc4849eae0dd22f79a1214da3453fcdd06c05...,SK,0,335,206,110,129,10,7,28,...,44,10,11,116,219,10,12,37,14,4
3,8a951fd81dc4849eae0dd22f79a1214da3453fcdd06c05...,NM,1,304,198,87,106,14,11,29,...,40,11,10,116,186,13,10,61,29,7
4,644d5090b896049b1351c7da557094dd9c48119022e7dd...,PA,0,255,173,83,82,11,7,28,...,53,16,30,109,141,12,7,19,16,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8461,ec692efb8fa838799813568da7c9014f816ef21679fdfc...,SW,1,356,244,131,112,14,10,30,...,55,26,24,119,229,14,14,49,8,11
8462,2da19cdc91d7ddf46e58c77f4e17624878c3ccf6a0ed52...,BL,0,391,249,111,142,14,9,27,...,71,20,19,144,230,11,12,48,4,9
8463,2da19cdc91d7ddf46e58c77f4e17624878c3ccf6a0ed52...,GE,1,319,205,64,114,12,12,57,...,78,19,20,148,170,8,7,51,12,8
8464,3aa5e3e21301815c8a83198b1d24ec5f5364b01a1e0c46...,SW,1,305,195,88,110,9,5,42,...,62,13,16,115,189,7,9,51,11,7


In [25]:

def subtract_home_away_stats(df):
    # Ensure that the data contains both home and away records for each game
    if 'Game_Hashkey' not in df.columns or 'Home_flg' not in df.columns:
        raise ValueError("DataFrame must contain 'Game_Hashkey' and 'Home_flg' columns")

    # Define the columns to subtract
    stats_columns = ['Disposals','Kicks','Marks','Handballs','Goals','Behinds','HitOuts','Tackles','Rebounds'
                         ,'Inside50','Clearances','Clangers','Frees','FreesAgainst','ContestedPossessions','UncontestedPossessions'
                         ,'ContestedMarks','MarksInside50','OnePercenters','Bounces','GoalAssists']

    # Initialize a list to store the result
    result_list = []

    # Group by 'Game_Hashkey' to get each game
    grouped = df.groupby('Game_Hashkey')

    for game_hash, group in grouped:
        # Find the home and away teams
        home_team = group[group['Home_flg'] == 1]
        away_team = group[group['Home_flg'] == 0]

        # Check that both home and away records exist for the game
        if home_team.empty or away_team.empty:
            continue

        # Perform the subtraction for each stat column
        differences = {}
        differences['Game_Hashkey'] = game_hash
        # differences['Home_Team'] = home_team['Team'].values[0]
        # differences['Away_Team'] = away_team['Team'].values[0]

        for col in stats_columns:
            differences[f"{col}_diff"] = home_team[col].values[0] - away_team[col].values[0]

        # Append the result to the list
        result_list.append(differences)

    # Convert the list to a DataFrame
    result_df = pd.DataFrame(result_list)
    return result_df

# Example usage:
# df is your input dataframe
# result = subtract_home_away_stats(df)
# print(result)


In [26]:
df_out = subtract_home_away_stats(df)

In [27]:
df_out

,Game_Hashkey,Disposals_diff,Kicks_diff,Marks_diff,Handballs_diff,Goals_diff,Behinds_diff,HitOuts_diff,Tackles_diff,Rebounds_diff,...,Clangers_diff,Frees_diff,FreesAgainst_diff,ContestedPossessions_diff,UncontestedPossessions_diff,ContestedMarks_diff,MarksInside50_diff,OnePercenters_diff,Bounces_diff,GoalAssists_diff
0,0014f5fd3c2f22e67a3684f6369b054beb14a4e6881f69...,61,39,35,22,4,-4,-14,29,-9,...,15,-3,3,-4,65,2,5,8,-3,8
1,0032ef175866c0f244f5b428be33b5ebf890646cdcde37...,-5,8,-24,-13,4,4,24,7,-14,...,-19,16,-16,14,-25,-1,0,17,0,-2
2,0057301a85fc320c8d859a601cc6a908b7e8e163143fc1...,88,29,5,59,19,7,4,-3,-3,...,-5,4,-4,18,75,-4,9,18,8,13
3,0064050165fa8166030b29892332a20767802bb3c35900...,3,-47,-24,50,-7,-3,-15,-18,2,...,16,0,0,19,-1,-1,-8,12,2,-3
4,0077cbe044197bd7145d4514159804a2b7c985bca11a09...,-5,12,4,-17,-5,-2,16,16,11,...,-6,7,-7,-3,-8,4,3,-8,2,-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4130,ffd51df3bf2753ab6c97faafc3725324f83e6d35b6225c...,-16,-34,-22,18,-2,-2,-19,15,13,...,20,-7,7,-15,-6,4,2,-7,0,0
4131,ffd6be5d8caa851e369de64537f5d6e37b9f155074c28f...,15,-37,-7,52,-11,-7,11,9,-4,...,1,3,-3,-16,32,-7,-10,-1,-10,-4
4132,ffe07677224f974e017dbdd6c0f4779034ffbce6b674e5...,67,37,26,30,3,6,18,9,-17,...,-8,3,-4,12,61,5,5,2,2,2
4133,ffe85092571594a2dc330609a9c07c9288fd80461586ef...,-37,-1,31,-36,-13,-6,-16,-14,16,...,-3,10,-10,-19,-8,2,-12,-7,-1,-8


In [28]:
def hashdiff_row(row):
    combined = (str(row['Disposals_diff']) + str(row['Kicks_diff']) + str(row['Marks_diff']) + str(row['Handballs_diff']) + str(row['Goals_diff'])
                + str(row['Behinds_diff'])  + str(row['HitOuts_diff']) + str(row['Tackles_diff']) +str(row['Rebounds_diff']) + str(row['Inside50_diff'])
                 + str(row['Clearances_diff']) +str(row[ 'Clangers_diff']) + str(row['Frees_diff'])   + str(row['FreesAgainst_diff']) 
                 + str(row[ 'ContestedPossessions_diff']) + str(row['UncontestedPossessions_diff'])   + str(row['ContestedMarks_diff']) 
                 + str(row['MarksInside50_diff']) + str(row['OnePercenters_diff']) + str(row['Bounces_diff']) + str(row['GoalAssists_diff']))
     # Generate a hash using SHA-256
    return hashlib.sha256(combined.encode()).hexdigest()

df_out['Hashdiff']=df_out.apply(hashdiff_row, axis=1)
df_out['Record_Source']='ScrapedPlayerStatistics'

df_out=df_out[['Game_Hashkey','Disposals_diff','Kicks_diff','Marks_diff','Handballs_diff','Goals_diff','Behinds_diff','HitOuts_diff'
                         ,'Tackles_diff','Rebounds_diff','Inside50_diff','Clearances_diff','Clangers_diff'
                         ,'Frees_diff','FreesAgainst_diff','ContestedPossessions_diff','UncontestedPossessions_diff'
                         ,'ContestedMarks_diff','MarksInside50_diff','OnePercenters_diff','Bounces_diff','GoalAssists_diff'
                         ,'Hashdiff','Record_Source']]

In [32]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
table_name='S_Game_Statistics'
connection=sqlite3.connect(database=data_base)
df_out.to_sql(table_name, connection, index=False, if_exists='append')
connection.close()

In [4]:
import pandas as pd
from datetime import datetime
import sqlite3
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
connection=sqlite3.connect(database=data_base)
query=f''' 
Select hg.year,hg.round,hg.team,hg.opponent 
       ,sgd.Ground,sgd.Team_Result
       ,sgs.Disposals_diff
       ,sgs.Kicks_diff
       ,sgs.Marks_diff
       ,sgs.Handballs_diff
       ,sgs.HitOuts_diff
       ,sgs.Tackles_diff
       ,sgs.Rebounds_diff
       ,sgs.Inside50_diff
       ,sgs.Clearances_diff
       ,sgs.Clangers_diff
       ,sgs.Frees_diff
       ,sgs.FreesAgainst_diff
       ,sgs.ContestedPossessions_diff
       ,sgs.UncontestedPossessions_diff
       ,sgs.ContestedMarks_diff
       ,sgs.MarksInside50_diff
       ,sgs.OnePercenters_diff
       ,sgs.Bounces_diff
       ,sgs.GoalAssists_diff
from h_game hg
join S_Game_Details sgd
on   hg.Game_hashkey=sgd.game_hashkey
join s_Game_statistics sgs
on   hg.game_hashkey=sgs.game_hashkey
--where hg.year=2024
--and hg.round='GF'
'''
df_feature=pd.read_sql(query,connection)
connection.close()
df_feature

,Year,Round,Team,Opponent,Ground,Team_Result,Disposals_diff,Kicks_diff,Marks_diff,Handballs_diff,...,Clangers_diff,Frees_diff,FreesAgainst_diff,ContestedPossessions_diff,UncontestedPossessions_diff,ContestedMarks_diff,MarksInside50_diff,OnePercenters_diff,Bounces_diff,GoalAssists_diff
0,2003,R01,CW,RI,M.C.G.,1,-3,5,10,-8,...,-1,-13,13,-19,8,-6,-2,4,4,7
1,2003,R01,NM,SK,M.C.G.,1,-31,-8,-23,-23,...,-4,1,-1,0,-33,3,-2,24,15,3
2,2003,R01,WC,PA,Subiaco,1,80,45,-1,35,...,1,14,-14,47,41,1,4,22,-6,1
3,2003,R01,SW,CA,Stadium Australia,1,90,56,54,34,...,3,-3,3,-5,87,7,17,12,8,9
4,2003,R01,BL,ES,Gabba,1,-23,-9,-15,-14,...,2,-2,2,-3,-27,6,7,11,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4130,2024,SF,PA,HW,Adelaide Oval,1,-15,21,-2,-36,...,9,2,-2,16,-39,-2,2,0,-3,-1
4131,2024,SF,GW,BL,Sydney Showground,0,22,2,4,20,...,9,3,-3,6,21,3,7,4,4,4
4132,2024,PF,SW,PA,S.C.G.,1,75,69,61,6,...,-8,2,-2,4,64,7,6,12,4,5
4133,2024,PF,GE,BL,M.C.G.,0,-72,-44,-47,-28,...,7,-1,1,4,-60,-3,-5,3,8,-1
